# New multi-image annotation code blocks

1. Initialization, label loading
2. Image filepath loading
3. Annotation
4. Label exporting

In [195]:
# Annotate images in batches of N
n = 10
labels = {
    'keep':      ['no', 'yes'], 
    'swift':     ['no', '1', '2', '3+'], 
    'blurry':    ['no', 'yes'], 
    'chimney':   ['no', 'yes'], 
    'antennae':  ['no', 'yes'],  
    'non-swift': ['no', 'crow', 'seagull', 'other'],
}

# Load existing annotations, or create empty .csv file if one does not already exist
label_filepath = f"{parent_dir}/labels_{'_'.join(list(labels.keys()))}.csv"

import os
import pandas as pd

if not os.path.exists(label_filepath):
    new_dfObj = pd.DataFrame(columns=["filename", "src_vid", "segment_id"] + list(labels.keys()))
    new_dfObj.to_csv(label_filepath)
    
dfObj = pd.read_csv(label_filepath)

In [196]:
# Load list of segment image filepaths
from pathlib import Path
from glob import glob
import re
from natsort import natsorted
from operator import itemgetter

parent_dir = Path("../data").resolve()
input_dir = parent_dir / "images"
frame_path_list = sorted(glob(str(input_dir/"**"/"*.png"), recursive=True))

p = re.compile(r'^.*"(.+)"_(\d+_\d+_\d+).png$')

seg_tups = [(m.group(0), m.group(1), m.group(2))  for m in
            [p.match(s) for s in frame_path_list] if m is not None
            and Path(m.group(0)).name not in dfObj["filename"].values]
seg_tups = natsorted(seg_tups, itemgetter(2))

print("Segments to annotate: {}".format(len(seg_tups)))

Segments to annotate: 10735


In [170]:
# Create annotations for set of images
from multi_label_pigeon import multi_label_annotate
from IPython.display import display, Image

if len(seg_tups) < n:
    end = len(seg_tups)  
else:
    end = n
start = 0  

annotations = multi_label_annotate(
    seg_tups[start:end],
    options=labels,
    display_fn=lambda seg_tup: display(Image(seg_tup[0], width=200, height=200))
    )

HTML(value='0 examples annotated, 11 examples left')

keep


swift


blurry


chimney


antennae


non-swift


Output()

Annotation done.


In [187]:
# Add new rows to dataframe
new_rows = []
for item, label_vals in annotations.items():
    label_vals = {key: val[0] for key, val in label_vals.items() if type(val) is list}
    row = pd.Series({"filename":Path(item[0]).name, "src_vid": item[1], "segment_id":item[2], **label_vals})
    
    if row["filename"] not in dfObj["filename"].values:
        new_rows.append(row)
dfObj = pd.concat([dfObj, pd.DataFrame(new_rows)])


# Sort and export
dfObj = dfObj.set_index("filename")
dfObj = dfObj.reindex(natsorted(dfObj.index))
dfObj = dfObj.reset_index(level="filename")
dfObj.to_csv(label_filepath, index=False)

# Older annotation code block when mutually exclusive labelling was used

Kept in case it needs to be returned to

In [7]:
# Annotate images in batches of N (mutually exclusive labels)
n = 1000
labels = ['swift', 'non-swift', "! blurry"]


# Actually label the images
from pigeon import annotate
from IPython.display import display, Image

if len(seg_tups) < n:
    end = len(seg_tups)  
else:
    end = n
start = 0  

annotations = annotate(
    seg_tups[start:end],
    options=labels,
    display_fn=lambda seg_tup: display(Image(seg_tup[0], width=200, height=200))
    )

HTML(value='0 examples annotated, 1001 examples left')

Output()